In [2]:
from google.colab import auth
auth.authenticate_user()

In [3]:
from google.cloud import bigquery

# Define your project and dataset
project_id = "group-5-448704"
mrt_dataset_id = "football_dataset_mrt"
stg_dataset_id = "football_dataset_stg"
int_dataset_id = "football_dataset_int"
raw_dataset_id = "football_dataset_raw"

# Initialize BigQuery client
client = bigquery.Client(project=project_id)

In [11]:
dataset = bigquery.Dataset(f"{client.project}.{mrt_dataset_id}")
dataset.location = "us-central1"
client.create_dataset(dataset, exists_ok=True)

Dataset(DatasetReference('group-5-448704', 'football_dataset_mrt'))

##Q1: How does stadium play an advantage?

###How does homefield lead to win loss by stadium?

In [20]:
query_1 = f"""
CREATE OR REPLACE TABLE `{project_id}.{mrt_dataset_id}.home_field_advantage_by_stadium` AS
SELECT
    Stadiums.stadium_name,
    COUNT(CASE WHEN Spreadspoke_Scores_Updated.score_home > Spreadspoke_Scores_Updated.score_away THEN 1 END) AS home_wins,
    COUNT(CASE WHEN Spreadspoke_Scores_Updated.score_home < Spreadspoke_Scores_Updated.score_away THEN 1 END) AS home_losses
FROM
    `{project_id}.{int_dataset_id}.Spreadspoke_Scores_Updated` AS Spreadspoke_Scores_Updated
JOIN
    `{project_id}.{int_dataset_id}.Stadiums` AS Stadiums
ON
    Spreadspoke_Scores_Updated.stadium_name = Stadiums.stadium_name
GROUP BY
    Stadiums.stadium_name;
"""

client.query(query_1.format(project_id=project_id, dataset_id=mrt_dataset_id)).result()



###How many touchdowns does a stadium have?

In [ ]:
query_10 = f"""
CREATE OR REPLACE TABLE `{project_id}.{mrt_dataset_id}.stadium_tds` AS
SELECT
    Spreadspoke_Scores_Updated.stadium_name,
    MAX(yearly_player_data.total_tds) AS total_tds
FROM
    `{project_id}.{int_dataset_id}.yearly_player_data` AS yearly_player_data
JOIN
    `{project_id}.{int_dataset_id}.Team_Identification` AS Team_Identification
ON
    yearly_player_data.team = Team_Identification.team_id
JOIN
    `{project_id}.{int_dataset_id}.Spreadspoke_Scores_Updated` AS Spreadspoke_Scores_Updated
ON
    Team_Identification.team_name= Spreadspoke_Scores_Updated.team_home
GROUP BY
    stadium_name
ORDER BY
    total_tds DESC;
"""

client.query(query_10.format(project_id=project_id, dataset_id=mrt_dataset_id)).result()



###How does homefield lead to win loss by team?

In [23]:
query_2 = f"""
CREATE OR REPLACE TABLE `{project_id}.{mrt_dataset_id}.home_field_advantage_by_team` AS
SELECT
    Stadiums.stadium_name,
    Spreadspoke_Scores_Updated.team_favorite_id,
    COUNT(CASE WHEN Spreadspoke_Scores_Updated.score_home > Spreadspoke_Scores_Updated.score_away THEN 1 END) AS home_wins,
    COUNT(CASE WHEN Spreadspoke_Scores_Updated.score_home < Spreadspoke_Scores_Updated.score_away THEN 1 END) AS home_losses
FROM
    `{project_id}.{int_dataset_id}.Spreadspoke_Scores_Updated` AS Spreadspoke_Scores_Updated
JOIN
    `{project_id}.{int_dataset_id}.Stadiums` AS Stadiums
ON
    Spreadspoke_Scores_Updated.stadium_name = Stadiums.stadium_name
JOIN
    `{project_id}.{int_dataset_id}.team_win_loss_tie_record` AS team_win_loss_tie_record
ON
    Spreadspoke_Scores_Updated.team_favorite_id = team_win_loss_tie_record.team
GROUP BY
    Stadiums.stadium_name, Spreadspoke_Scores_Updated.team_favorite_id;
"""

client.query(query_2.format(project_id=project_id, dataset_id=mrt_dataset_id)).result()



##Q2: Which players are most impactful?

###How many pass touchdowns does a player have?

In [21]:
query_3 = f"""
CREATE OR REPLACE TABLE `{project_id}.{mrt_dataset_id}.pass_td_player` AS
SELECT
    yearly_player_data.name,
    Team_Identification.team_name,
    yearly_player_data.pass_td,
FROM
    `{project_id}.{int_dataset_id}.Team_Identification` AS Team_Identification
JOIN
    `{project_id}.{int_dataset_id}.yearly_player_data` AS yearly_player_data
ON
    Team_Identification.team_id = yearly_player_data.team
"""

client.query(query_3.format(project_id=project_id, dataset_id=mrt_dataset_id)).result()


###How many receiving yards does a player have?

In [35]:
query_4 = f"""
CREATE OR REPLACE TABLE `{project_id}.{mrt_dataset_id}.receiving_yards` AS
SELECT
    yearly_player_data.name,
    Team_Identification.team_name,
    yearly_player_data.receiving_yards,
FROM
    `{project_id}.{int_dataset_id}.Team_Identification` AS Team_Identification
JOIN
    `{project_id}.{int_dataset_id}.yearly_player_data` AS yearly_player_data
ON
    Team_Identification.team_id = yearly_player_data.team
"""

client.query(query_4.format(project_id=project_id, dataset_id=mrt_dataset_id)).result()


###What is the height and weight of the top players?

In [17]:
query_9 = f"""
CREATE OR REPLACE TABLE `{project_id}.{mrt_dataset_id}.height_weight_ypg` AS
SELECT
    yearly_player_data.name,
    renamed_2024_player_predictions.height,
    renamed_2024_player_predictions.weight,
    AVG(yearly_player_data.ypg) AS avg_ypg
FROM
    `{project_id}.{int_dataset_id}.yearly_player_data` AS yearly_player_data
JOIN
    `{project_id}.{int_dataset_id}.renamed_2024_player_predictions` AS renamed_2024_player_predictions
ON
    yearly_player_data.college = renamed_2024_player_predictions.college
GROUP BY
    name, height, weight
ORDER BY
    avg_ypg DESC;
"""

client.query(query_9.format(project_id=project_id, dataset_id=mrt_dataset_id)).result()



##Q3: How did different conferences perform?

###What is the win loss rating by conference?

In [56]:
query_5 = f"""
CREATE OR REPLACE TABLE `{project_id}.{mrt_dataset_id}.conference_win_loss` AS
SELECT
    Team_Conference.team_conference,
    team_name,
    COUNT(CASE WHEN Spreadspoke_Scores_Updated.score_home > Spreadspoke_Scores_Updated.score_away THEN 1 END) AS home_wins,
    COUNT(CASE WHEN Spreadspoke_Scores_Updated.score_home < Spreadspoke_Scores_Updated.score_away THEN 1 END) AS home_losses
FROM
    `{project_id}.{int_dataset_id}.Spreadspoke_Scores_Updated` AS Spreadspoke_Scores_Updated
JOIN
    `{project_id}.{int_dataset_id}.Team_Conference` AS Team_Conference
ON
    Spreadspoke_Scores_Updated.team_home = Team_Conference.team_name
GROUP BY
    team_conference, team_name;
"""

client.query(query_5.format(project_id=project_id, dataset_id=mrt_dataset_id)).result()



###What is the average player rating by conference?

In [8]:
query_6 = f"""
CREATE OR REPLACE TABLE `{project_id}.{mrt_dataset_id}.player_conference_rating` AS
SELECT
    Team_Conference.team_conference,
    yearly_player_data.name,
    AVG(yearly_player_data.pass_ypg) AS avg_pass_ypg,
    AVG(yearly_player_data.rush_ypg) AS avg_rush_ypg
FROM
    `{project_id}.{int_dataset_id}.yearly_player_data` AS yearly_player_data
JOIN
    `{project_id}.{int_dataset_id}.Team_Identification` AS Team_Identification
ON
    yearly_player_data.team = Team_Identification.team_id
JOIN
    `{project_id}.{int_dataset_id}.Team_Conference` AS Team_Conference
ON
    Team_Identification.team_name = Team_Conference.team_name
GROUP BY
    team_conference, name
ORDER BY
    avg_pass_ypg DESC, avg_rush_ypg DESC;
"""

client.query(query_6.format(project_id=project_id, dataset_id=mrt_dataset_id)).result()



###What is the average bad play rating by conference?

In [12]:
query_7 = f"""
CREATE OR REPLACE TABLE `{project_id}.{mrt_dataset_id}.player_conference_badplay_rating` AS
SELECT
    Team_Conference.team_conference,
    yearly_player_data.name,
    AVG(yearly_player_data.pass_fumble_lost) AS avg_pass_fumble_lost,
    AVG(yearly_player_data.reception_fumble_lost) AS avg_reception_fumble_lost,
    AVG(yearly_player_data.run_fumble_lost) AS avg_run_fumble_lost
FROM
    `{project_id}.{int_dataset_id}.yearly_player_data` AS yearly_player_data
JOIN
    `{project_id}.{int_dataset_id}.Team_Identification` AS Team_Identification
ON
    yearly_player_data.team = Team_Identification.team_id
JOIN
    `{project_id}.{int_dataset_id}.Team_Conference` AS Team_Conference
ON
    Team_Identification.team_name = Team_Conference.team_name
GROUP BY
    team_conference, name
ORDER BY
    avg_pass_fumble_lost DESC, avg_reception_fumble_lost DESC, avg_run_fumble_lost DESC;
"""

client.query(query_7.format(project_id=project_id, dataset_id=mrt_dataset_id)).result()

